# Data Preprocessing

Wikipedia is an encyclopedia that covers a large amount of diverse topics. All articles are created, corrected and updated by individuals. The goal is to correctly document as many topics as possible by collecting the knowledge of a large number of people. However, some articles stand out due to their completeness, scope and presentation, and for this they are marked with the distinction of the Excellent Article. 

As part of the Natural Language Processing lecture, a classification of Wikipedia articles is to be carried out as a sub-task of an assignment with the goal of being able to identify excellent articles. This notebook contains the code to accomplish this goal and is structured as follows:

- [1. Imports](article_classification.ipynb#1-imports)
- [2. Enrichment of the Data](article_classification.ipynb#2-check-data-availability)
	- [2.1 Add Subscription Details](#thema1)
	- [2.2 Download Thumnails and add Path to jpg to the df](#thema2)
- [3. Data Cleaning](article_classification.ipynb#3-data-processing)
	- [4.1 Define Data Cleaning](#thema1)
	- [4.2 Define String Encoding](#thema2)
	- [4.3 Apply Functions](#thema3)
- [4. Export df]()


## 1. Imports
Import the requiered libraties into the notebook.
If some libraries are not installed, you can use the `requierements.txt` and run
```
$ pip install -r requirements.txt
```
in the terminal.

In [1]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-20 16:41:14.307133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Loading

In [3]:
df = pd.read_csv("../Data/CAvideos.csv")
df = df[:500]

# Enrichment of the Data

## Add Subscription Details for Channel Details

Subscribing to a channel on YouTube can make it easier for you to watch multiple videos from the same channel. Therefore the Subscription count is a measure for the YouTubers reach. [1] According to range of the audiance that can be reached through a video, the subscription count is an important feature when forecasting the potential views of a video. 

In the following the `YouTube Data API v3` from `www.googleapis.com` is going to be used to add the subscription coun of the channel to the df.

[1] https://tuberanker.com/blog/what-are-the-benefits-of-subscribing-to-a-youtube-channel (04.06.23)

In [6]:
# The following youtube_authenticate function is available under this url: https://www.thepythoncode.com/article/using-youtube-api-in-python#google_vignette
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_883243427026-gmpvuoto8cd0gsl6djl80sesaa6b5jo0.apps.googleusercontent.com.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [7]:
# See the Youtube API Documentation for more insights: https://developers.google.com/youtube/v3/docs/subscriptions/list?hl=de&apix=true

def get_channel_details(youtube, **kwargs):

    request = youtube.channels().list(
        part="statistics", # just get stats
        **kwargs
    )
    response = request.execute()

    return response

In [8]:
# get subscription count and save it to the df

from tqdm import tqdm

for i in tqdm(range (0, len(df))):

    try:

        # get current channel Name
        channel_name = df.iloc[i]["channel_title"]

        # make API call to get channel infos
        response = get_channel_details(youtube, forUsername=channel_name)
        items = response.get("items")[0]
        statistics = items["statistics"]

        # get stats infos
        subscriberCount = statistics["subscriberCount"]

        # save subscriptions to df
        df.at[i,"subscriber_count"] = int(subscriberCount)

    except:

        df.at[i,"subscriber_count"] = np.NAN

100%|██████████| 40881/40881 [22:59<00:00, 29.63it/s]


## Download Thumnails and add Path to jpg to the df

Every YouTube video is represented by a thumbnail, a small image that, along with the title and channel, serves as the “cover” of the video. Thumbnails that are interest- ing and well-framed attract viewers, while those that are confusing and low-quality encourage viewers to click else- where. As a testament to the important of a good thumb- nail, 90% of the most successful YouTube videos have cus- tom thumbnails [2]. YouTube uploaders without the time or skills to create a custom thumbnail, however, must pick one of 3 frames automatically chosen from the video. Our mission is to improve this frame selection process and help uploaders select high quality frames that will attract viewers to their channel.

http://cs231n.stanford.edu/reports/2017/pdfs/710.pdf
[2] YouTube Creator Academy. Lesson: Make click- able thumbnails. https://creatoracademy. youtube.com/page/lesson/thumbnails# yt-creators-strategies-5.

In [11]:
df = df[:100]

In [4]:
# load thumbnail and save picture path in df
import urllib.request
from tqdm import tqdm
import os

df_w_jgp = df.copy()

def load_and_save_thumbnail(url, iterator):
    try:

        # get unique thumbnail id 
        splitted_words = url.split("/")
        thumbnail_id = splitted_words[4]

        # create filename to save thumbnail
        file_path = f"../Data/thumbnails/{thumbnail_id}.jpg"
        df_w_jgp.at[iterator,"thumbnail_link"] = thumbnail_id

        # check if thumbnail is already downloaded
        if os.path.exists(file_path):
            pass
        else:
            # download and save thumbnail 
            urllib.request.urlretrieve(url, file_path)

    except urllib.error.URLError as e:

        df_w_jgp.at[iterator,"thumbnail_link"] = np.NAN
        # print(f"An error occurred: {e}")
        return None

for i in tqdm(range (0, len(df))):

    # get thumbnail url
    url = df.iloc[i]["thumbnail_link"]

    # safe thumnail and write path into df
    load_and_save_thumbnail(url, i)  

100%|██████████| 500/500 [00:08<00:00, 58.34it/s]


# Data Cleaning

## Define Data Cleaning

In [5]:
def data_cleaning(df):
    
    # Handling Missing Values

    if df.isnull().any().any() or df.isna().any().any():

        num = df.isnull().sum().sum() + df.isna().sum().sum() # Count Missing Values

        print(">>>",num, "Missing Values are beeing handeled")

        #df.dropna() # Drop rows with missing values
        #df.fillna(value) # Fill missing values with a specific value

    else:
        print(">>> No Missing Values detected")
        
    # Remove Duplicate

    if df.duplicated().any():

        num = df.duplicated().sum() # Count Duplicates

        print(">>>",num, "Duplicates are beeing handeled")

        df.drop_duplicates()

    else:
        print(">>> No Duplicates detected")

    # Handling Outliers


    # Drop Unnececary Columns

    df = df.drop(['video_id', 'trending_date', 'thumbnail_link', 'publish_time'], axis=1)

    # Convert Data Types 

    # print( df.dtypes ) # check data types

    # Convert 'date_column' to datetime
    #df['trending_date'] = pd.to_datetime(df['trending_date'], format='%Y-%m-%d %H:%M:%S.%f')

    # Convert 'timestamp_column' to timestamp
    #df['publish_time'] = pd.to_datetime(df['publish_time'], infer_datetime_format=True)

    for column in df:

        # Check if column is of type bool
        if df[column].dtype == 'bool':
            df[column] = df[column].astype(int)

    # Check preprocessed data

    print(">>> Updated Data Types \n", df.dtypes)

    return df

## Define String Encoding

In [16]:
df_w_jgp

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,[5],"[1, 3, 2]","[146, 229, 13, 147, 55, 40, 230]",[460],10,"[2, 1, 223, 7, 19, 4]","[179, 575, 41, 406, 1686, 1687, 1688, 229]",17158579,787425,43420,125882,n1WpP7iowLc,False,False,False,"[4450, 43, 1283, 616, 11, 463, 205, 2124, 28, ..."
1,[6],"[1, 3, 2]","[231, 232, 233, 114, 234]",[461],23,"[2, 1, 59, 7, 7, 4]","[762, 456, 168, 168, 313, 763, 867, 1689, 137,...",1014651,127794,1688,13030,0dBIkQ4Mz1M,False,False,False,"[856, 535, 13, 1066, 12, 2257, 2637, 129, 231,..."
2,[7],"[1, 3, 2]","[148, 235, 236, 237, 238, 239, 240, 241]","[108, 109]",23,"[2, 1, 167, 60, 24, 4]","[457, 408, 353, 354, 121, 764, 457, 408, 86, 3...",3191434,146035,5339,8181,NaN,False,False,False,"[54, 63, 1310, 30, 1068, 1, 76, 68, 7, 6, 14, ..."
3,[8],"[1, 3, 2]","[16, 242, 21, 243, 244]",[163],24,"[2, 1, 81, 10, 90, 4]","[737, 1698, 1029, 461, 64, 767, 33, 1699, 1700...",2095828,132239,1989,17518,d380meD0W0M,False,False,False,"[51, 197, 274, 362, 13, 607, 837, 96, 982, 37,..."
4,"[19, 20]","[1, 3, 2]","[245, 476, 149, 18, 42, 15]","[111, 197]",10,"[2, 1, 1250, 61, 5, 4]","[2516, 409, 768, 2517, 81, 2518, 31, 1335, 31,...",33523622,1634130,21082,85067,2Vv-BfVoq4g,False,False,False,"[6507, 7, 1191, 6508, 1301, 529, 6509, 7, 6510..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,[494],"[1, 5, 2]","[2374, 317, 28, 7, 6, 2375, 134, 919]","[2724, 1077]",22,"[2, 1, 1274, 36, 17, 4]","[557, 1457, 238, 557, 153, 1457, 557, 49, 557,...",52205,164,27,93,NaN,False,False,False,"[6410, 13, 1711, 242, 9284, 1744, 2686, 5, 36,..."
496,[95],"[1, 5, 2]","[817, 818, 13, 819, 820, 821]","[289, 1099, 1100, 20, 1101]",25,"[2, 1, 148, 32, 20, 4]","[1304, 1305, 3168, 277, 1303, 554, 1216, 530, ...",21839,28,18,27,lSozbwZmq74,False,False,False,"[3, 8627, 2239, 8628, 129, 1373, 156, 19, 3, 3..."
497,[108],"[1, 5, 2]","[868, 45, 365, 869, 870, 84, 871, 872, 873, 874]",[1113],24,"[2, 1, 283, 25, 49, 4]","[998, 999, 238, 1233, 1236, 1235, 3246, 1234, ...",338908,25407,228,3184,vIVaLtJmyPs,False,False,False,"[3, 1378, 1132, 36, 2201, 1644, 116, 1506, 45,..."
498,[5],"[1, 5, 2]","[146, 229, 13, 147, 55, 40, 230]",[460],10,"[2, 1, 223, 7, 19, 4]","[179, 575, 41, 406, 1686, 1687, 1688, 229]",22702386,869304,50018,123235,n1WpP7iowLc,False,False,False,"[4450, 43, 1283, 616, 11, 463, 205, 2124, 28, ..."


In [14]:
def encoding(df_cont):# encode remaining objects

    tokenizer = Tokenizer(
        num_words=10000,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    )

    for column in df:

        # Check if column is of type object
        if df[column].dtype == 'object' and column != "video_id":

            df[column] = df[column].astype(str)

            print(f">>> {column} is of type object and will be encoded")

            tokenizer.fit_on_texts(df[column])

            # Get the word index
            word_index = tokenizer.word_index

            # Convert texts to sequences
            sequences = tokenizer.texts_to_sequences(df[column])

            df_cont[column] = sequences

            df_cont

            # split sequences into seperate columns

        else:
            pass
    
    for column in df_cont:

            # Check if column is of type object
            if df_cont[column].dtype == 'object':

                # Create separate columns for each element in the list
                df_expanded = pd.DataFrame(df_cont[column].to_list(), columns=[f"{column}_{i+1}" for i in range(df_cont[column].str.len().max())])
                
                # Concatenate expanded columns with the original DataFrame
                df_cont = pd.concat([df_cont.drop(column, axis=1), df_expanded], axis=1)
                
    return df_cont
        

## Apply Functions and save final DataFrame as CSV

In [21]:
import os
import pandas as pd

folder_path = "../Data"  # Replace with the path to your folder
final_df = []  # List to store the DataFrames

iterator = 0
# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name != 'final_dataset.csv':  # Process only CSV files
        file_path = os.path.join(folder_path, file_name)

        print(file_path)

        # Read the CSV file and create a DataFrame
        df = pd.read_csv(file_path, encoding='latin-1')

        cleaned_df = data_cleaning(df)

        encoded_df = encoding(cleaned_df)

        encoded_df["country"] = iterator # file_name[:3]

        iterator += 1

        # Add the DataFrame to the list
        final_df.append(encoded_df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(final_df, ignore_index=True)

# Print the combined DataFrame
print(combined_df)


../Data/dataset_w_jpg.csv
>>> 29236 Missing Values are beeing handeled
>>> 221 Duplicates are beeing handeled
>>> Updated Data Types 
 title                      object
channel_title              object
category_id               float64
tags                       object
views                     float64
likes                     float64
dislikes                  float64
comment_count             float64
comments_disabled          object
ratings_disabled           object
video_error_or_removed     object
description                object
dtype: object
>>> video_id is of type object and will be encoded
>>> trending_date is of type object and will be encoded
>>> title is of type object and will be encoded
>>> channel_title is of type object and will be encoded
>>> publish_time is of type object and will be encoded
>>> tags is of type object and will be encoded
>>> thumbnail_link is of type object and will be encoded
>>> comments_disabled is of type object and will be encoded
>>> ratings_d

# Export final df

In [13]:
df_w_jgp.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [15]:
encoded_df = encoding(df_w_jgp)

>>> video_id is of type object and will be encoded
>>> trending_date is of type object and will be encoded
>>> title is of type object and will be encoded
>>> channel_title is of type object and will be encoded
>>> publish_time is of type object and will be encoded
>>> tags is of type object and will be encoded
>>> description is of type object and will be encoded


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [16]:
# Replace NaN values with 0
#df_w_jgp= df_w_jgp.fillna(0)

df_cnn = df_w_jgp[['views', 'thumbnail_link']]

# Save final df as csv
csv_path = "../Data/final_dataset_cnn.csv"
df_cnn.to_csv(csv_path, index=False)

In [22]:
# Replace NaN values with 0
combined_df= combined_df.fillna(0)

# Save final df as csv
csv_path = "../Data/final_dataset.csv"
combined_df.to_csv(csv_path, index=False)